In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

/mnt/task_runtime/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.45it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


What is your name?assed
What is your name?assed
What is your name?assed
What is your name?assed
What is your name?assed
What is your name?


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (ro

In [ ]:
from datasets import load_dataset

ds = load_dataset("qwedsacf/competition_math")

Generating train split: 100%|██████████| 12500/12500 [00:00<00:00, 377565.89 examples/s]


In [23]:
# Get a geometry level 5 problem from the dataset
geometry_level5 = ds['train'].filter(lambda x: x.get('type') == 'Geometry' and x.get('level') == 'Level 5')
problem = geometry_level5[5]


# Format the problem for the model
messages = [
    {"role": "user", "content": f"Solve this geometry problem without using any external tools. Put your solution in \\boxed{...} format.\n\n Here is the problem:\n\n{problem['problem']}"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=2048)
response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

print("Problem:")
print(problem['problem'])
print("\n" + "="*80 + "\n")
print("Model Response:")
print(response)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Problem:
The center of a circle has coordinates $(6,-5)$. The circle is reflected about the line $y=x$. What are the $x,y$ coordinates of the center of the image circle? State the $x$ coordinate first.


Model Response:
To solve this problem, we need to reflect the center of the circle, which has coordinates (6, -5), about the line y = x. Reflecting a point (x, y) about the line y = x results in the point (y, x). Therefore, reflecting the point (6, -5) about the line y = x gives us the point (-5, 6). So, the coordinates of the center of the image circle are (-5, 6).

Here is the Python code to confirm this:

```python
# Define the original coordinates of the circle's center
original_x, original_y = 6, -5

# Reflect the coordinates about the line y = x
reflected_x, reflected_y = original_y, original_x

# Print the reflected coordinates
print(f"({reflected_x}, {reflected_y})")
```

When you run this code, it will output: (-5, 6), which confirms our solution.


In [24]:
problem['solution']

'The center of the image circle is simply the center of the original circle reflected over the line $y=x$. When reflecting over this line, we swap the $x$ and $y$ coordinates. Thus, the image center is the point $\\boxed{(-5, 6)}$.'

In [26]:
import datasets
generated_dataset_level5 = datasets.load_from_disk('data/math_solutions_dataset_20000/')

In [31]:
generated_dataset_level5

[0]

In [33]:
# Reload the dataset to ensure it's loaded correctly
from datasets import load_from_disk

generated_dataset_level5 = load_from_disk('data/math_solutions_dataset_20000/')

# Check the dataset structure
print(f"Dataset type: {type(generated_dataset_level5)}")
print(f"Dataset length: {len(generated_dataset_level5)}")
if hasattr(generated_dataset_level5, 'features'):
    print(f"Dataset features: {generated_dataset_level5.features}")
    print(f"\nFirst example:")
    print(generated_dataset_level5[0])
else:
    print(f"Error: Dataset is not a proper Dataset object. It's a {type(generated_dataset_level5)}")
    print(f"First item: {generated_dataset_level5[0] if len(generated_dataset_level5) > 0 else 'empty'}")


Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 553
Dataset features: {'problem': Value('string'), 'ground_truth': Value('string'), 'solution': Value('string')}

First example:
{'problem': 'Square ABCD has its center at $(8,-8)$ and has an area of 4 square units. The top side of the square is horizontal. The square is then dilated with the dilation center at (0,0) and a scale factor of 2. What are the coordinates of the vertex of the image of square ABCD that is farthest from the origin? Give your answer as an ordered pair.', 'ground_truth': 'With the center of dilation at the origin and a scale factor of 2, all the coordinates of square $ABCD$ are twice the coordinates of its preimage. The preimage has an area of 4 square units, so its side length is 2 units. Since the center of the preimage is at $(8, -8)$, the four vertices of the preimage are at $(7, -9), (7, -7), (9, -7)$ and $(9, -9)$. The point $(9, -9)$ is the farthest from the origin on the preimage, so 

In [ ]:
# Load the fine-tuned model
from transformers import AutoTokenizer, AutoModelForCausalLM

finetuned_tokenizer = AutoTokenizer.from_pretrained("./qwen_finetuned")
finetuned_model = AutoModelForCausalLM.from_pretrained("./qwen_finetuned")
finetuned_model.to(device)


In [ ]:
generated_dataset_level5[0]

{'problem': 'Square ABCD has its center at $(8,-8)$ and has an area of 4 square units. The top side of the square is horizontal. The square is then dilated with the dilation center at (0,0) and a scale factor of 2. What are the coordinates of the vertex of the image of square ABCD that is farthest from the origin? Give your answer as an ordered pair.',
 'ground_truth': 'With the center of dilation at the origin and a scale factor of 2, all the coordinates of square $ABCD$ are twice the coordinates of its preimage. The preimage has an area of 4 square units, so its side length is 2 units. Since the center of the preimage is at $(8, -8)$, the four vertices of the preimage are at $(7, -9), (7, -7), (9, -7)$ and $(9, -9)$. The point $(9, -9)$ is the farthest from the origin on the preimage, so the point farthest from the origin on the image of square $ABCD$ is $\\boxed{(18, -18)}.$',
 'solution': "\\begin{intermediatederivation}\nThe square has area \\(4\\), so its side length is \\(s=\\sq